<a href="https://colab.research.google.com/github/Makoto-OKADA-OPU/tanimura_regotaiwa/blob/main/predictMaskedPoses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語母語話者の英語対訳コーパスに対する深層言語モデルを用いた単語予測の分析と評価

岡田 真

修正：2024.07  
初出：2023.10


## 概説
「インタラクションと対話」[^1] 第 4 章 (岡田担当章) で用いた
実験用 Python スクリプトです．  

具体的には課題達成対話データから作成された文字化データに含まれる「ポーズ (沈黙)」箇所を深層言語モデルにより推定して分析しています．  

その際の Python スクリプトになります．

スクリプトの各部に説明を加えています．そちらを読みながら動作確認できます．

[^1]:[谷村緑，仲本康一郎，吉田悦子：インタラクションと対話，開拓社，2024．](https://www.kaitakusha.co.jp/book/book.php?c=2401)






## 初期化
ライブラリの読み込みやインストールをします．

In [ ]:
#!/usr/bin/env python
#-*- coding:utf-8 -*-

# 必要なモジュールの読み込み
import os # オペレーティングシステム (OS) インターフェース Python から OS にアクセスするためのライブラリ
import glob # Unix 形式のパス名のパターン解析用ライブラリ Python からファイルやディレクトリにアクセスするためのライブラリ
import pandas as pd # データ解析ライブラリ


In [ ]:
# BERT を使う準備
# Google Colaboratory で transformers ライブラリをインストールして tokenizer と model の用意をする．
!pip install transformers # transformers のインストール

import torch # PyTorch 深層学習用ライブラリの読み込み
from transformers import BertTokenizer, BertForPreTraining # BERT のトークナイザと事前学習モデルの読み込み

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')

## 関数
処理を簡単にするために関数を作成しています．

In [ ]:
# [MASK] で推測された上位の単語を得る関数
# tokens: トークン
# k: 上位 k 個取得 (デフォルト 20 個)
def predictMaskedWord(tokens, k=20):
  predicted_tokens = None
  results_tokens = []

  # '[MASK]' トークンが含まれるデータで予測する
  if '[MASK]' in tokens: # [MASK] トークンを含んでいる場合に予測する
    # [MASK] トークンの位置を見つけてその番号を保存する
    # 1 つの文に複数の [MASK] トークンが含まれている場合があるので，
    # 文中のすべての [MASK] トークンの位置の番号を保存する
    masked_indecis = [i for i, x in enumerate(tokens) if x == '[MASK]']

    # BERT で予測する
    # トークン ID を取得して，BERT で処理をする形式に変更してから BERT にデータを入力する．
    ids = tokenizer.convert_tokens_to_ids(tokens)
    ids = torch.tensor(ids).reshape(1, -1) # バッチサイズ 1 の形に変形する

    # BERT に入力して各トークンの分散表現ベクトルを得る
    with torch.no_grad():
      output = model(ids)

    # 出力結果を検証する
    # prediction_logits は [文の数，文中のトークンの数，全ての語彙の数] という形式のテンソルになっている
    # 各文の各トークンに対してすべての語彙との距離が格納されている
    # つまり各文の各単語について，全てのトークンとの分散表現ベクトルの距離の近さが取れる

    pred = output.prediction_logits # masked language modeling の出力結果 (分散表現ベクトル)
    pred = pred[0] # [文の数 (ここでは 1), トークンの数, 全ての語彙の数] という形なので最初の [1] を取り除くために代入し直す

    # すべての [MASK] トークンについて他の全てのトークンとの距離のうち
    # 上位から k 個のトークンの情報を得る
    # torch.topk モジュールは入力されたテンソルの上位 k 個についてその値と ID を返す
    # ここでは値は必要ないので _ で無視して，ID だけを得ている
    for i in masked_indecis:
      _, pred_idxs = torch.topk(pred[i], k) # 上位 k 個の ID を得る
      predicted_tokens = tokenizer.convert_ids_to_tokens(pred_idxs.tolist()) # ID からトークン (文字列) を得る
      results_tokens.append(predicted_tokens) # トークンを格納する

  return results_tokens

In [ ]:
# 取得した予測トークンの集計をする関数
# リスト中の全てのトークンを走査して，その出現頻度を数え上げる
# listlisttokens トークンのリストのリスト
# cnt トークンの出現頻度
from collections import Counter

def countMaskedWords(listlisttokens, cnt):
  for listtoken in listlisttokens:
    for token in listtoken:
      cnt[token] += 1


## 処理本体
プログラム本体です．文字化データのファイルを読み込んで，[MASK] 部分に入るトークンを推定します．  


文字化データを Google Drive 上に置いて，そこから読み込む場合，事前にデータを Google Drive 上に格納しておき，下記のコードを実行してプログラム中で自分の Google Drive 上のフォルダにアクセスできるようにする必要があります．  
下記のコードを実行すると Google Drive へのアクセスを許可するかどうか聞かれますので，許可をして処理を進めます．

In [ ]:
# Google Drive をプログラム中から読み込めるようにする (マウントする)
from google.colab import drive
drive.mount('/content/drive')

処理したい文字化データのファイルリストを得ます．  
変数 ```filepath``` には文字化データのある Google Drive のディレクトリの完全パスを設定してください．  
例えば Google Drive の MyDrive に mojikadata というディレクトリがあって，その中にあるのであれば，```filepath``` には ```"/content/drive/MyDrive/mojikadata/"``` という完全パスを入力します．  

このプログラムでは，文字化データは Excel 形式のファイルという前提で作成しています．

今回はサンプルデータを Google Drive の Colab Notebooks の中に WordEstimationUsingMaskedLearningModel というディレクトリを作り，この Colab Notebook を保存して，その同じディレクトリに sampledata というディレクトリを作って，そこに "sampledata.xlsx" という Excel 形式のサンプルファイルを置いています．  
このファイルには書籍で例として使ったデータ 2 件が入っています．  

実際に自分でこの Notebook を使う場合はデータのあるディレクトリの完全パスを自分の環境に合わせた設定に変更してください．

In [ ]:
# ファイルリストを得ます．
# 具体的には対象となる文字化データファイルを探してその完全パスをすべて保存します．
#
# 今回はサンプルデータを Google Drive の Colab Notebooks の中に
# WordEstimationUsingMaskedLearningModel というディレクトリを作り，
# この Colab Notebook を保存して，その同じディレクトリに sampledata という
# ディレクトリを作って，そこに "sampledata.xlsx" という Excel 形式の
# サンプルファイルを置いています．
# このファイルには書籍で例として使ったデータ 2 件が入っています．
# 実際に自分で使う場合は自分の環境に合わせた設定に変更してください．
filepath = "/content/drive/MyDrive/Colab Notebooks/WordEstimationUsingMaskedLearningModel/sampledata/"

# ファイルパス内の該当ファイルを取得
import glob
files = glob.glob(filepath + '**/*.xlsx', recursive=True) # ディレクトリの中を
files.sort()
for f in files:
  print(f) # 読み込んだデータファイルの確認

文字化データが Excel 形式で格納されている前提で，今回は pandas という Python のデータ構造やデータ解析のライブラリを使って文字化データの読み込みや [MASK] 部分の推定とその推定結果の集計をしていきます．

In [ ]:
# データファイルで「ポーズ (沈黙)」部分を [MASK] して推測する
# ライブラリの読み込み
from tqdm.auto import tqdm # 処理の進行状況表示用ライブラリ
import numpy as np # 数値計算用ライブラリ
from collections import Counter, defaultdict # 汎用ユーティリティライブラリからトークンカウント用にモジュールを読み込む

# トークンカウントの準備
cnt = Counter()
l_n_sentence = defaultdict(lambda:0)
l_n_masked_sentence = defaultdict(lambda:0)


for f in tqdm(files): # ファイルごとに処理
  print(f) # 処理するファイル名を表示
  basefilename, ext = os.path.splitext(os.path.basename(f))

  # pandas.read_excel モジュールを使って読み込んでデータフレームに格納する
  # 先頭行をヘッダとしないために header オプションに None を渡している
  # これにより読み込んだ最初の行から使われて，0, 1, 2,... と番号付けられる
  # Excel のシートごとに処理をするために sheet_name オプションを使う
  # ここではすべてのシートを扱うために None を渡している
  df = pd.read_excel(f, header=None, sheet_name=None)

  # Excel のシートごとに処理する
  for k in df.keys():
    # 「ポーズ (沈黙)」部分を '[MASK]' で置換する．
    # ファイルのデータは df に格納される．
    # 0 列目は発話者の情報など，1 列目に発話内容の文字化データが格納されている
    import re
    df[k][2] = df[k][1].map(lambda x: re.sub(re.compile(r"\([0-9]+\)"), "[MASK]", x), na_action='ignore') # 文字化データに「ポーズ」があれば [MASK] に置き換えて，2 列目に格納する
    df[k][3] = df[k][2].map(tokenizer.tokenize, na_action='ignore') # 2 列目のデータをトークンに分割して，3 列目に格納する

    # 置換してトークン化したデータで Masked Word Prediction をして，予測単語の上位 20 語を得て，4 列目に保存する
    df[k][4] = df[k][3].map(predictMaskedWord, na_action='ignore')

    # 結果を集計する
    # 文字化データの行数を数える
    l_n_sentence[basefilename] = len(df[k][1].dropna())

    # 予測単語を集計する
    for x in df[k][4].dropna(how='all').to_numpy():
      if len(x) > 0:
        l_n_masked_sentence[basefilename] += 1
        countMaskedWords(x, cnt)

# 結果をファイルに出力する
# シートごとに結果を出力していく
with pd.ExcelWriter(filepath+basefilename+'.pred'+ext) as writer:
  for k in df.keys():
    Writer = df[k].to_excel(writer, index=False, header=False, sheet_name=k)

# 集計結果を画面に出力して確認する
for k, v in cnt.items():
  print(k,v)


In [ ]:
# 結果を画面に出力して確認する
# すべての文について
print('Total Sentence')
sum = 0
for k, v in l_n_sentence.items():
  print(k, v)
  sum += v
print("total: {}".format(sum))

# すべての [MASK] を含む文について
print('Total Sentance with [MASK]')
sum = 0
for k, v in l_n_masked_sentence.items():
  print(k, v)
  sum += v
print("total: {}".format(sum))

# 予測したトークンについて
print('Total Predicted Tokens')
print('total: {}'.format(len(cnt)))
for k, v in cnt.items():
  print(k,v)

In [ ]:
# トークンと出現頻度の情報を CSV ファイルに出力する
import csv

with open(filepath+"listtokencount.csv", mode="w", newline='') as f:
  writer = csv.writer(f)
  for d in cnt.most_common():
    writer.writerow(d)


In [ ]:
# トークンの情報を画面に表示する
for d in cnt.most_common():
  print(d)

## Transformers の Pipelines を利用した方法
Hugging Face の Transformers に Pipelines ライブラリがあります．  
こちらはモデルを使った推論が簡単にできるライブラリです．  
書籍では紹介していませんが，ここで簡単に説明します．

詳しい説明は [Hugging Face の説明](https://huggingface.co/docs/transformers/main_classes/pipelines) を参照してください．

この Pipelines ライブラリの pipeline モジュールを使うことで MASK されたトークンの推定や文書分類や文書生成を Transformers にアップロードされている様々なモデルを使って実行することができます．

モデルの内部構造まで踏み込んだ開発などではない際にモデルを使った処理をしたいという場合は便利なモジュールとなっています．

In [ ]:
from transformers import pipeline

# pipeline の準備
# 今回は MASK されたトークンの推定として "fill-mask" を，
# 使用するモデルとして "bert-base-uncased" を，
# 上位から出力する個数として 10 (これは数値なので "" なし) を指定しています
pipe = pipeline("fill-mask", "bert-base-uncased", top_k=10)

# MASK されたトークンの推定処理
# 結果が result に出力されます
# "This is [MASK] . " という文の [MASK] 部分に入るトークンを予測して出力が返ってきます．
result = pipe("This is [MASK] .")

# 結果の表示
# スコア (類似度)，トークン ID，トークン文字列，MASK を補完した文字列が出力されています
print(f"k: {len(result)}")
print("Predicted Tokens")
for t in result:
  print(t)


上記のプログラムの [MASK] 部分の推定をこの pipeline を利用した方法に置き換えることができます．